# Using ``heudiconv``-style heuristics to get accurate BIDS 

BIDS curation is not a straightforward process and can be very tricky when non-standard names are used at the scanner.

The python package ``heudiconv`` approaches this problem by having users write a "heuristic" python script that assigns scans to BIDS names based on properties in their dicom headers. When a user runs ``heudiconv`` they supply the path to dicoms, the subject ID, session ID and the path to a python script that implements their heuristic. The software then groups the dicoms based on studyUID and other shared properties in their headers, uses the user's script to determine what their names should be in BIDS, then runs dcm2niix and renames the files.

The first and last parts of this process are already fully automated in flywheel. Therefore, one possible solution for getting accurate BIDS curation would be to build a system that can query a session's acquisitions and adjust their metadata according to the user-provided heuristic.

## Heuristic files

At the end of the day, the heuristic file loops over acquisitions and sends their metadata through some user-provided logic. The scans themselves are provided as part of a ``SeqInfo`` object. The scans in the SeqInfo expose attributes to the user that can be used to classify those scans. Here is the complete list:

```python
seqinfo_fields = [
    'total_files_till_now',  # 0
    'example_dcm_file',      # 1
    'series_id',             # 2
    'dcm_dir_name',          # 3
    'unspecified2',          # 4
    'unspecified3',          # 5
    'dim1', 'dim2', 'dim3', 'dim4', # 6, 7, 8, 9
    'TR', 'TE',              # 10, 11
    'protocol_name',         # 12
    'is_motion_corrected',   # 13
    'is_derived',            # 14
    'patient_id',            # 15
    'study_description',     # 16
    'referring_physician_name', # 17
    'series_description',    # 18
    'sequence_name',         # 19
    'image_type',            # 20
    'accession_number',      # 21
    'patient_age',           # 22
    'patient_sex',           # 23
    'date',                  # 24
    'series_uid'             # 25
]
```

### The ``infotodict`` function and its output

This function has to be implemented in the user's heuristic script. It takes a ``SeqInfo`` object as its only argument and returns a dictionary that maps a formatting tuple to lists of scan ids. The formatting tuple is defined by the user in the script, based on a boilerplate that heudiconv provides in its template. For reference, here is the function that creates the keys used in the ``infotodict`` output:

```python
def create_key(template, outtype=('nii.gz',), annotation_classes=None):
    if template is None or not template:
        raise ValueError('Template must be a valid format string')
    return template, outtype, annotation_classes
```

The return value is hashable and can be used as a key. The user creates one key per scan type at the beginning of their implementation of ``infotodict``. Here is an example from the GRMPY_822831 project in the BBL group:

```python
def infotodict(seqinfo):
    """Heuristic evaluator for determining which runs belong where

    allowed template fields - follow python string module:

    item: index within category
    subject: participant id
    seqitem: run number during scanning
    subindex: sub index within group
    """

    last_run = len(seqinfo)
    
    # Create Keys
    t1w = create_key(
       'sub-{subject}/{session}/anat/sub-{subject}_{session}_T1w')
    t2w = create_key(
       'sub-{subject}/{session}/anat/sub-{subject}_{session}_T2w')
    dwi = create_key(
       'sub-{subject}/{session}/dwi/sub-{subject}_{session}_acq-multiband_dwi')
    
    # Field maps
    b0_phase = create_key(
       'sub-{subject}/{session}/fmap/sub-{subject}_{session}_phasediff')
    b0_mag = create_key(
       'sub-{subject}/{session}/fmap/sub-{subject}_{session}_magnitude')
    pe_rev = create_key(
        'sub-{subject}/{session}/fmap/sub-{subject}_{session}_acq-multiband_dir-j_epi')
    # fmri scans
    rest_mb = create_key(
       'sub-{subject}/{session}/func/sub-{subject}_{session}_task-rest_acq-multiband_bold')
    rest_sb = create_key(
       'sub-{subject}/{session}/func/sub-{subject}_{session}_task-rest_acq-singleband_bold')
    fracback = create_key(
       'sub-{subject}/{session}/func/sub-{subject}_{session}_task-fracback_acq-singleband_bold')
    face = create_key(
       'sub-{subject}/{session}/func/sub-{subject}_{session}_task-face_acq-singleband_bold')

    # ASL scans
    asl = create_key(
       'sub-{subject}/{session}/asl/sub-{subject}_{session}_asl')
    asl_dicomref = create_key(
       'sub-{subject}/{session}/asl/sub-{subject}_{session}_acq-ref_asl')
    m0 = create_key(
       'sub-{subject}/{session}/asl/sub-{subject}_{session}_m0')
    mean_perf = create_key(
       'sub-{subject}/{session}/asl/sub-{subject}_{session}_mean-perfusion')
```


The object that gets returned can then be defined as 

```python
    info = {t1w:[], t2w:[], dwi:[], b0_phase:[], 
            b0_mag:[], pe_rev:[], rest_mb:[], rest_sb:[],
            fracback:[], asl_dicomref:[], face:[], asl:[],
            m0:[], mean_perf:[]}

```

Finally, the ``SeqInfo`` object is looped over and the scan_ids are added to the lists in the ``info`` dictionary. Here is part of an example loop:

```python

    def get_latest_series(key, s):
        if len(info[key]) == 0:
            info[key].append(s.series_id)
        else:
            if info[key] < s.series_id:
                info[key] = [s.series_id]

    for s in seqinfo:
        protocol = s.protocol_name.lower()
        if "mprage" in protocol:
            get_latest_series(t1w,s)
        elif "t2_sag" in protocol:
            get_latest_series(t2w,s)
        elif "b0map" in protocol and "M" in s.image_type:
            info[b0_mag].append(s.series_id)
        elif "b0map" in protocol and "P" in s.image_type:
            info[b0_phase].append(s.series_id)
        elif "topup_ref" in protocol:
            get_latest_series(pe_rev, s)
        elif "dti_multishell" in protocol and not s.is_derived:
            get_latest_series(dwi, s)
                elif s.series_description.endswith("_ASL"):
            get_latest_series(asl, s)
        elif protocol.startswith("asl_dicomref"):
            get_latest_series(asl_dicomref, s)
        elif s.series_description.endswith("_M0"):
            get_latest_series(m0, s)
        elif s.series_description.endswith("_MeanPerf"):
            get_latest_series(mean_perf, s)

        elif "fracback" in protocol:
            get_latest_series(fracback, s)
        elif "face" in protocol:
            get_latest_series(face,s)
        elif "rest" in protocol:
            if "MB" in s.image_type:
                get_latest_series(rest_mb,s)
            else:
                get_latest_series(rest_mb,s)
    return info
```

Note that the ``get_latest_series`` function is used to address the situation where the a scan is interrupted and the interrupted scan is then followed by a complete version of the scan. It is common to have both the aborted and complete scan in a session, so this will always use the latest version of the scan. It is also possible to append multiple series_ids to the list and their ordering is formatted with their position in the list (like ``run-{item:02d}`` in the format string). 

It is then straightforward to take the ``info`` dictionary and update the bids fields in flywheel. 


## Goals

If possible, it would be a major achievement to write a gear or python package that can take a heuristic file that works with ``heudiconv`` and have it work without any changes in flywheel. Since it won't have to do dicom conversion or actual file renaming, the only actual work to do is to:

  1. Write an implementation of the ``SeqInfo`` class that is built from flywheel scan info
  2. Write an implementation of the file renaming that updates scan metadata on flywheel
  
## Reference materials

Here is the full ``grmpy_heuristic.py`` file:

```python
import os


def create_key(template, outtype=('nii.gz',), annotation_classes=None):
    if template is None or not template:
        raise ValueError('Template must be a valid format string')
    return template, outtype, annotation_classes


def infotodict(seqinfo):
    """Heuristic evaluator for determining which runs belong where

    allowed template fields - follow python string module:

    item: index within category
    subject: participant id
    seqitem: run number during scanning
    subindex: sub index within group
    """

    last_run = len(seqinfo)
    
    # Create Keys
    t1w = create_key(
       'sub-{subject}/{session}/anat/sub-{subject}_{session}_T1w')
    t2w = create_key(
       'sub-{subject}/{session}/anat/sub-{subject}_{session}_T2w')
    dwi = create_key(
       'sub-{subject}/{session}/dwi/sub-{subject}_{session}_acq-multiband_dwi')
    
    # Field maps
    b0_phase = create_key(
       'sub-{subject}/{session}/fmap/sub-{subject}_{session}_phasediff')
    b0_mag = create_key(
       'sub-{subject}/{session}/fmap/sub-{subject}_{session}_magnitude')
    pe_rev = create_key(
        'sub-{subject}/{session}/fmap/sub-{subject}_{session}_acq-multiband_dir-j_epi')

    # fmri scans
    rest_mb = create_key(
       'sub-{subject}/{session}/func/sub-{subject}_{session}_task-rest_acq-multiband_bold')
    rest_sb = create_key(
       'sub-{subject}/{session}/func/sub-{subject}_{session}_task-rest_acq-singleband_bold')
    fracback = create_key(
       'sub-{subject}/{session}/func/sub-{subject}_{session}_task-fracback_acq-singleband_bold')
    face = create_key(
       'sub-{subject}/{session}/func/sub-{subject}_{session}_task-face_acq-singleband_bold')

    # ASL scans
    asl = create_key(
       'sub-{subject}/{session}/asl/sub-{subject}_{session}_asl')
    asl_dicomref = create_key(
       'sub-{subject}/{session}/asl/sub-{subject}_{session}_acq-ref_asl')
    m0 = create_key(
       'sub-{subject}/{session}/asl/sub-{subject}_{session}_m0')
    mean_perf = create_key(
       'sub-{subject}/{session}/asl/sub-{subject}_{session}_mean-perfusion')

    info = {t1w:[], t2w:[], dwi:[], b0_phase:[], 
            b0_mag:[], pe_rev:[], rest_mb:[], rest_sb:[],
            fracback:[], asl_dicomref:[], face:[], asl:[],
            m0:[], mean_perf:[]}

    def get_latest_series(key, s):
        if len(info[key]) == 0:
            info[key].append(s.series_id)
        else:
            if info[key] < s.series_id:
                info[key] = [s.series_id]

    for s in seqinfo:
        protocol = s.protocol_name.lower()
        if "mprage" in protocol:
            get_latest_series(t1w,s)
        elif "t2_sag" in protocol:
            get_latest_series(t2w,s)
        elif "b0map" in protocol and "M" in s.image_type:
            info[b0_mag].append(s.series_id)
        elif "b0map" in protocol and "P" in s.image_type:
            info[b0_phase].append(s.series_id)
        elif "topup_ref" in protocol:
            get_latest_series(pe_rev, s)
        elif "dti_multishell" in protocol and not s.is_derived:
            get_latest_series(dwi, s)

        elif s.series_description.endswith("_ASL"):
            get_latest_series(asl, s)
        elif protocol.startswith("asl_dicomref"):
            get_latest_series(asl_dicomref, s)
        elif s.series_description.endswith("_M0"):
            get_latest_series(m0, s)
        elif s.series_description.endswith("_MeanPerf"):
            get_latest_series(mean_perf, s)

        elif "fracback" in protocol:
            get_latest_series(fracback, s)
        elif "face" in protocol:
            get_latest_series(face,s)
        elif "rest" in protocol:
            if "MB" in s.image_type:
                get_latest_series(rest_mb,s)
            else:
                get_latest_series(rest_mb,s)
    return info
```

And here is an example of the fields that go into the ``SeqInfo`